In [1]:
from FNN import FNN
from Layer import Layer
from Neuron import Neuron
import numpy as np
from ActivFunctions import  *
from InitFunctions import  *
from SuppFunctions import  *

Neuron Class functionality testing

In [2]:
inputTest = np.array([1.0,1.0])

In [3]:
#zero initialization
t0NeuronZero = Neuron(2,identity)
print(t0NeuronZero.weight_vector.shape)
t0NeuronZero.forward(inputTest)

(1, 3)


[array([[0.]]), array([[0.]])]

In [4]:
#random initialization
t0NeuronRandom = Neuron(2,identity,method_ini="Random")
print(t0NeuronRandom.weight_vector.shape)
t0NeuronRandom.forward(inputTest)

(1, 3)


[array([[0.70627357]]), array([[0.70627357]])]

In [5]:
#initialization from vector
vector = np.array([1.0,2.0,3.5])
print(vector)
print(vector.shape)
t1Neuron = Neuron(vector,identity)
print(t1Neuron.weight_vector.shape)
t1Neuron.forward(inputTest)

[1.  2.  3.5]
(3,)
(1, 3)


[array([[6.5]]), array([[6.5]])]

In [6]:
#initialization from row vector(2D array)
vector_row = vector[np.newaxis,:]
print(vector_row)
print(vector_row.shape)
t2Neuron = Neuron(vector_row,identity)
print(t2Neuron.weight_vector.shape)
t2Neuron.forward(inputTest)

[[1.  2.  3.5]]
(1, 3)
(1, 3)


[array([[6.5]]), array([[6.5]])]

In [7]:
#initialization from column vector(2D array)
vector_column = vector[:,np.newaxis]
print(vector_column)
print(vector_column.shape)
t3Neuron = Neuron(vector_column,identity)
print(t3Neuron.weight_vector.shape)
t3Neuron.forward(inputTest)

[[1. ]
 [2. ]
 [3.5]]
(3, 1)
(1, 3)


[array([[6.5]]), array([[6.5]])]

In [8]:
#array input
t0NeuronRandom = Neuron(2,identity,method_ini="Random")
inputTest = np.array([[1.0,1.0],[1.0,1.0]])
print(inputTest.shape)
print(inputTest)
print(t0NeuronRandom.weight_vector.shape)
t0NeuronRandom.forward(inputTest)

(2, 2)
[[1. 1.]
 [1. 1.]]
(1, 3)


[array([[0.62823325, 0.62823325]]), array([[0.62823325, 0.62823325]])]

In [9]:
#wrong activ function
vector = np.array([1.0,2.0,3.5])
errorNeuron = Neuron(vector,1)

NotImplementedError: 

In [ ]:
#wrong input function
vector = np.array(["1.0","2.0","3.5"])
errorNeuron = Neuron(vector,identity)

NotImplementedError: 

In [ ]:
#wrong input (too small)
vector = np.array([1.0,2.0,3.5])
neuronTest = Neuron(vector,identity)
inputTestErrorSmall = np.array([1.0])
neuronTest.forward(inputTestErrorSmall)

NotImplementedError: 

In [ ]:
#wrong input (too big)
vector = np.array([1.0,2.0,3.5])
neuronTest = Neuron(vector,identity)
inputTestErrorBig = np.array([[1.0],[1.0],[1.0],[1.0]])
print(inputTestErrorBig)
neuronTest.forward(inputTestErrorBig)

[[1.]
 [1.]
 [1.]
 [1.]]


NotImplementedError: 

Layer class implementation testing

In [ ]:
#initialization by number of dim in vector
dim_layer = np.array([4,5])
testLayer = Layer(dim_layer,identity,method_ini="Random")
testLayer.weights_array
testLayer.activ_functions

[<function ActivFunctions.identity(x)>,
 <function ActivFunctions.identity(x)>,
 <function ActivFunctions.identity(x)>,
 <function ActivFunctions.identity(x)>]

In [ ]:
#forward test pass
inputTestVect = np.array([[1.0],[1.0],[1.0],[1.0],[1.0]])
inputTestArray = np.array([[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]])

print(testLayer.forward(inputTestVect))
print(testLayer.forward(inputTestArray))

[array([[2.93741261],
       [3.16570166],
       [4.12556719],
       [3.06488001]]), array([[2.93741261],
       [3.16570166],
       [4.12556719],
       [3.06488001]])]
[array([[2.93741261, 4.92025167],
       [3.16570166, 6.10149423],
       [4.12556719, 7.70806968],
       [3.06488001, 5.19384362]]), array([[2.93741261, 4.92025167],
       [3.16570166, 6.10149423],
       [4.12556719, 7.70806968],
       [3.06488001, 5.19384362]])]


FNN class implementation testing

In [12]:
#initialization by number of dim in vector
dim_layer = np.array([5,5,3,2,1])
testNet = FNN(dim_layer,identity,method_ini="Random")
print(testNet.weights_list[0])
print(testNet.weights_list[1])
print(testNet.weights_list[2])
print(testNet.weights_list[3])
print(testNet.activ_functions_list[0])
print(testNet.activ_functions_list[1])
print(testNet.activ_functions_list[2])
print(testNet.activ_functions_list[3])
#forward test pass
inputTestVect = np.array([[1.0],[1.0],[1.0],[1.0],[1.0]])
inputTestArray = np.array([[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0],[1.0,2.0]])

print(testNet.forward(inputTestVect))
print(testNet.forward(inputTestArray))


[[0.28156461 0.66980571 0.61900519 0.09558239 0.64158846 0.83980218]
 [0.67968198 0.37002793 0.91374895 0.57998655 0.41853405 0.85563371]
 [0.07669919 0.91433914 0.37862469 0.13313974 0.29647039 0.08852724]
 [0.95751846 0.68891225 0.68969802 0.88200442 0.59904858 0.15260945]
 [0.27435503 0.18519506 0.66283938 0.40497699 0.09623416 0.12438847]]
[[0.68030577 0.49412327 0.96877788 0.96247331 0.6735594  0.11385461]
 [0.00288413 0.41420213 0.28138484 0.10421633 0.28902314 0.64024188]
 [0.00594837 0.69361562 0.81496139 0.91313827 0.5032143  0.55503246]]
[[0.34204032 0.32508896 0.88554357 0.15301817]
 [0.29299083 0.54682188 0.0127216  0.27421699]]
[[0.46546055 0.19271379 0.14741595]]
[<function identity at 0x00000140650BA5C0>, <function identity at 0x00000140650BA5C0>, <function identity at 0x00000140650BA5C0>, <function identity at 0x00000140650BA5C0>, <function identity at 0x00000140650BA5C0>]
[<function identity at 0x00000140650BA5C0>, <function identity at 0x00000140650BA5C0>, <function i